# Значения интерфейсного типа

В Go значение интерфейсного типа, или по-другому **значение интерфейса**, состоит из двух компонент:конкретного типа и значения этого типа. Эти компоненты называются **динамическим типом** и **динамическим значением** интерфейса соответственно.

Рассмотрим следующий пример, в котором значение интерфейса w принимает по очереди несколько значений(первое и последнее значения являются одинаковыми):

In [1]:
import (
    "io"
    "fmt"
    "os"
    "bytes"
)

var w io.Writer
w = os.Stdout
w = new(bytes.Buffer)
w = nil

Сначала идёт объявление значения интерфейса:

In [ ]:
var w io.Writer

В Go всегда происходит инициализация переменных и интерфейсы не являются исключением. **Нулевое значение** интерфейса это - такое значение интерфейса, у которого компоненты динамического типа и динамического значения оба равны **nil**. 

Равенство значения интерфейса nil определяется на **основе динамического типа**: если динамический тип равен nil, то и значение интерфейса равно nil(т.е w равно nil) и наоборот. Проверить равенство значения интерфейса nil можно простым сравнением с nil:

In [2]:
import "fmt"
fmt.Sprint(w == nil)

true


Вызов какого-либо метода nil интерфейса вызывает панику:

In [ ]:
// Выполнение этого кода вызовёт панику.
w.Write([]byte("Hello"))

Далее идёт присваивание w значения типа \*os.File:

In [ ]:
w = os.Stdout

Присваивания такого рода неявно преобразовывают конкретный тип к интерфейсному типу. Компонента динамического типа значения интерфейса w теперь хранит описание типа \*os.File, а компонента динамического значения содержит копию значения типа os.Stdout, который является указателем на переменную типа os.File. Вызов метода Write для значения интерфейса, который содержит \*os.File, приводит к вызову (\*os.File).Write.

Далее идёт присваивание w значения типа \*bytes.Buffer:

In [ ]:
w = new(bytes.Buffer)

Теперь динамическим типом w становится \*bytes.Buffer, а динамическим значением - указатель на только что созданный буфер. На этот раз вызов w.Write приведёт к вызову (\*bytes.Buffer).Write.

И в конце происходит присваивание nil:

In [ ]:
w = nil

Это присваивание устанавливает равными nil и динамический тип и динамическое значение переменной w.

Значения интерфейса можно сравнивать между собой, используя == или !=. Два значения интерфейса равны, если они оба равны nil или если оба содержат один и тот же динамический тип и динамические значения равны друг другу, следуя правилу сравнения для этого типа:

In [3]:
var a interface{} = [2]int{1, 2}
var b interface{} = [2]int{1, 2}
fmt.Sprint(a == b) // Работает, т.к. массивы можно сравнивать между собой.

true


Однако если два значения интерфейса имеют одинаковый динамический тип, но значения этого типа не являются сравнимыми(как, например, слайсы), то сравнение вызовет панику:

In [ ]:
// Выполнение следующего кода вызовет панику.
var a interface{} = []int{1, 2}
var b interface{} = []int{1, 2}
fmt.Sprint(a == b)

Это поведение является **специфичным** для интерфейсов. Другие типо либо можно безопасно сравнивать, либо сравнение вызовет **ошибку компиляции**.

Динамический тип значения интерфейса можно узнать с помощью глагола %T пакета fmt:

In [4]:
var w io.Writer
fmt.Printf("%T\n", w)

w = os.Stdout
fmt.Printf("%T\n", w)

w = new(bytes.Buffer)
fmt.Sprintf("%T", w)

<nil>
*os.File
*bytes.Buffer


При работе с интерфейсами часто допускают **следующую ошибку**: проверяют значение интерфейса на равенство nil и если оно не равно nil, то вызывают какой-либо метод этого интерфейса. Такод код может привести к панике. Рассмотрим пример:

In [ ]:
func f(w io.Writer) {
    if w != nil {
        w.Write([]byte("Trash"))
    }
}

var buf *bytes.Buffer
// Выполнение следующей строки кода вызовет панику.
f(buf)

Здесь произошло следующее: переменная buf после объявления равна nil, т.е. она никуда не указывает, но она имеет тип \*bytes.Buffer. Этот тип удовлетворяет интерфейсу io.Writer, поэтому ошибки компиляции при вызове f не будет. При передаче аргумента buf функции f параметр w этой функции будет проинициализирован так: динамический тип будет равен \*bytes.Buffer, а динамическое значение nil. Как мы уже говорили чуть выше, равенство значения интерфейса nil проверяется на основе динамического типа. В нашем примере динамический тип не равен nil, поэтому и w тоже не равно nil. Вызов w.Write определяет, что необходимо вызвать метод Write для типа \*bytes.Buffer и в качестве получателя передать nil, т.к. именно nil'у равно динамическое значение w. Для некоторых типов(как, например, \*os.File) передача nil в качестве значения получателю метода не является ошибкой, но для типа \*bytes.Buffer это не так. Поэтому и происходит паника во время выполнения этого кода.